In [8]:
%run SetUpParams.ipynb

2024-08-13 01:18:55.590516
2004-08-13 01:18:55.590516


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [9]:
import pandas as pd 
import numpy as np

### Create dataframe to feed into model

In [10]:
final_df = pd.DataFrame()
# Next Day Open Price - Current Day Open Price
final_df[f'{target_stock}_change'] = target_stock_data['Open'].shift(-1) - target_stock_data['Open']
final_df[f'{target_stock}_lag'] = final_df[f'{target_stock}_change'].shift(1)
final_df['Close'] = target_stock_data['Close']
final_df['Price'] = target_stock_data['Open']
final_df.reset_index(inplace=True)
print(final_df)

           Date  ^GSPC_change  ^GSPC_lag        Close        Price
0    2004-08-13      1.570068        NaN  1064.800049  1063.229980
1    2004-08-16     14.539917   1.570068  1079.339966  1064.800049
2    2004-08-17      2.369995  14.539917  1081.709961  1079.339966
3    2004-08-18     13.460083   2.369995  1095.170044  1081.709961
4    2004-08-19     -3.940063  13.460083  1091.229980  1095.170044
...         ...           ...        ...          ...          ...
5028 2024-08-06     86.709961  55.279785  5240.029785  5206.419922
5029 2024-08-07    -40.560059  86.709961  5199.500000  5293.129883
5030 2024-08-08     62.090332 -40.560059  5319.310059  5252.569824
5031 2024-08-09     37.219727  62.090332  5344.160156  5314.660156
5032 2024-08-12           NaN  37.219727  5348.740234  5351.879883

[5033 rows x 5 columns]


#### Add European and US Stock Market Price Changes to final_df
Current Day Open Price - Previous Day Open Price

In [11]:
# EU Stock Market has already opened
for ticker in US_EU_market_dict.keys():
        data = US_EU_market_dict[ticker]
        # Current Day Open Price - Previous Day Open Price
        temp = (data['Open'] - data['Open'].shift(1)).reset_index()
        temp.rename(columns={'Open':ticker}, inplace=True)
        final_df = final_df.merge(temp, on='Date', how='left')

print(final_df)

           Date  ^GSPC_change  ^GSPC_lag        Close        Price  \
0    2004-08-13      1.570068        NaN  1064.800049  1063.229980   
1    2004-08-16     14.539917   1.570068  1079.339966  1064.800049   
2    2004-08-17      2.369995  14.539917  1081.709961  1079.339966   
3    2004-08-18     13.460083   2.369995  1095.170044  1081.709961   
4    2004-08-19     -3.940063  13.460083  1091.229980  1095.170044   
...         ...           ...        ...          ...          ...   
5028 2024-08-06     86.709961  55.279785  5240.029785  5206.419922   
5029 2024-08-07    -40.560059  86.709961  5199.500000  5293.129883   
5030 2024-08-08     62.090332 -40.560059  5319.310059  5252.569824   
5031 2024-08-09     37.219727  62.090332  5344.160156  5314.660156   
5032 2024-08-12           NaN  37.219727  5348.740234  5351.879883   

           ^IXIC        ^DJI      ^FCHI      ^GDAXI  
0            NaN         NaN        NaN         NaN  
1      -2.880005   11.239258   3.310059   -5.809814

#### Add Asian Stock Market Price Changes to final_df
Close Price - Open Price

In [12]:
# Asian Stock Market has already closed; one day ahead of US Stock Market
for ticker in asia_market_dict.keys():
        data = asia_market_dict[ticker]
        # Close Price - Open Price
        temp = (data['Close'] - data['Open']).reset_index()
        temp.rename(columns={0:ticker}, inplace=True)
        final_df = final_df.merge(temp, on='Date', how='left')

print(final_df)

           Date  ^GSPC_change  ^GSPC_lag        Close        Price  \
0    2004-08-13      1.570068        NaN  1064.800049  1063.229980   
1    2004-08-16     14.539917   1.570068  1079.339966  1064.800049   
2    2004-08-17      2.369995  14.539917  1081.709961  1079.339966   
3    2004-08-18     13.460083   2.369995  1095.170044  1081.709961   
4    2004-08-19     -3.940063  13.460083  1091.229980  1095.170044   
...         ...           ...        ...          ...          ...   
5028 2024-08-06     86.709961  55.279785  5240.029785  5206.419922   
5029 2024-08-07    -40.560059  86.709961  5199.500000  5293.129883   
5030 2024-08-08     62.090332 -40.560059  5319.310059  5252.569824   
5031 2024-08-09     37.219727  62.090332  5344.160156  5314.660156   
5032 2024-08-12           NaN  37.219727  5348.740234  5351.879883   

           ^IXIC        ^DJI      ^FCHI      ^GDAXI       ^AORD        ^HSI  \
0            NaN         NaN        NaN         NaN   -6.399902   12.469727   
1

In [13]:
# Different markets has different holidays
# Forward fill null values
final_df.ffill(inplace=True)
final_df.dropna(inplace=True)
# final_df.isna().sum()
final_df

,Date,^GSPC_change,^GSPC_lag,Close,Price,^IXIC,^DJI,^FCHI,^GDAXI,^AORD,^HSI,^N225,^NSEI
778,2007-09-17,-7.609985,0.290039,1476.650024,1484.239990,11.890137,20.560547,-16.479980,-17.530273,-23.299805,-378.080078,232.370117,-23.800293
779,2007-09-18,43.119995,-7.609985,1519.780029,1476.630005,-1.080078,-38.770508,-104.929688,-37.049805,-66.500000,73.019531,-235.690430,52.100098
780,2007-09-19,8.939941,43.119995,1529.030029,1519.750000,72.929932,337.430664,264.319824,169.069824,113.100098,43.281250,343.419922,182.100098
781,2007-09-20,-9.939941,8.939941,1518.750000,1528.689941,-6.309814,72.909180,2.810059,99.669922,14.299805,62.820312,-60.871094,12.699707
782,2007-09-21,7.000000,-9.939941,1525.750000,1518.750000,11.169922,-45.189453,-18.060059,-5.809570,-22.099609,183.818359,28.180664,84.599609
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5028,2024-08-06,86.709961,55.279785,5240.029785,5206.419922,548.830078,-319.972656,60.479980,-103.019531,30.700195,-213.960938,2598.130859,-197.298828
5029,2024-08-07,-40.560059,86.709961,5199.500000,5293.129883,360.950195,493.871094,1.870117,16.080078,23.000000,193.080078,967.269531,8.099609
5030,2024-08-08,62.090332,-40.560059,5319.310059,5252.569824,-214.041016,-289.710938,61.069824,77.400391,-26.600098,118.791016,185.359375,-131.550781
5031,2024-08-09,37.219727,62.090332,5344.160156,5314.660156,228.250000,467.679688,36.690430,156.849609,104.200195,-7.128906,-247.339844,-19.349609
